# Machine learning homework5
## 1. Preparation
### 1.1 Import required librarys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from libsvm.python import svm
from libsvm.python import svmutil

### 1.2 Read training and testing data

In [ ]:
x_train = np.loadtxt('data/X_train.csv', delimiter=',')
x_test = np.loadtxt('data/X_test.csv', delimiter=',')
y_train = np.loadtxt('data/Y_train.csv', delimiter=',')
y_test = np.loadtxt('data/Y_test.csv', delimiter=',')

### 1.3 Construct problem with training data

In [ ]:
problem = svmutil.svm_problem(y_train, x_train)

## 2. Compare different kernels
### 2.1 Train with linear kernel
Use default parameters and quiet mode

In [ ]:
linear_param = svmutil.svm_parameter('-t 0 -q')
linear_model = svmutil.svm_train(problem, linear_param)

### 2.2 Train with polynomial kernel
Use default parameters and quiet mode

In [ ]:
poly_param = svmutil.svm_parameter('-t 1 -q')
poly_model = svmutil.svm_train(problem, poly_param)

### 2.3 Train with RBF kernel
Use default parameters and quiet mode

In [ ]:
rbf_param = svmutil.svm_parameter('-t 2 -q')
rbf_model = svmutil.svm_train(problem, rbf_param)

### 2.4 Predict testing data with three trained model

In [ ]:
linear_pred = svmutil.svm_predict(y_test, x_test, linear_model)
poly_pred = svmutil.svm_predict(y_test, x_test, poly_model)
rbf_pred = svmutil.svm_predict(y_test, x_test, rbf_model)

### 2.5 Visualize each kernel's accuracy

In [ ]:
names = ['Linear', 'Polynomial', 'RBF']
acc = [linear_pred[1][0], poly_pred[1][0], rbf_pred[1][0]]
plt.figure()
plt.bar(names, acc, width=0.5)
plt.title('Accuracy of different kernels with default parameters')
plt.xlabel('Kernels')
plt.ylabel('Accuracy')
plt.show()